# DEM CRS/Scale Finder + Island Removal

This notebook combines two steps that worked:

1. **Auto-fit CRS and XY scaling** against a target WGS84 region
2. **Remove small offshore islands** from the DEM (mask positive-elevation blobs below an area threshold)

> You can edit paths and parameters in the first cell below.

In [2]:

# --- Configuration ---
from pathlib import Path

# Input Montserrat DEM points (XYZ ASCII: x y z) — change to your path
ASC_XYZ = Path("/Users/glennthompson/Dropbox/PROFESSIONAL/DATA/wadgeDEMs/original/DEM_10m_1999_xyz.asc")

# Output directory where candidates and figures are written
OUT_DIR = Path("/Users/glennthompson/Dropbox/PROFESSIONAL/DATA/wadgeDEMs/auto_crs_fit_v2")

# Target region in WGS84 lon/lat (minlon, maxlon, minlat, maxlat) to score overlap against
DEFAULT_REGION = (-62.255, -62.135, 16.66, 16.84)  # Montserrat

# XY scale factors to try (1.0 = as-is; 0.4 for 10m->4m mis-scale; 0.3048 if feet->meters in XY)
SCALE_FACTORS = [1.0, 0.4, 0.3048]

# If you want a PyGMT overlay PNG of the best candidate, leave True (requires GMT)
PLOT_OVERLAY = True

NODATA = -32768.0

ASC_XYZ, OUT_DIR


(PosixPath('/Users/glennthompson/Dropbox/PROFESSIONAL/DATA/wadgeDEMs/original/DEM_10m_1999_xyz.asc'),
 PosixPath('/Users/glennthompson/Dropbox/PROFESSIONAL/DATA/wadgeDEMs/auto_crs_fit_v2'))

In [3]:

from __future__ import annotations
from typing import Tuple, Optional, List
import numpy as np
import rasterio
from rasterio.crs import CRS
from rasterio.transform import Affine
from rasterio.warp import reproject, Resampling, calculate_default_transform

# Optional PyGMT overlay
try:
    import pygmt
except Exception:
    PLOT_OVERLAY = False

def _median_step(vals: np.ndarray) -> float:
    vals = np.unique(vals)
    steps = np.diff(np.sort(vals))
    return float(np.median(steps)) if steps.size else 1.0

def _grid_xyz(e: np.ndarray, n: np.ndarray, z: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    x_vals = np.unique(e)
    y_vals = np.unique(n)
    nx, ny = x_vals.size, y_vals.size
    x_index = np.searchsorted(x_vals, e)
    y_index = np.searchsorted(y_vals, n)
    Z = np.full((ny, nx), np.nan, dtype=float)
    Z[y_index, x_index] = z
    # Make north-up (row 0 = north)
    if y_vals.size > 1 and (y_vals[1] > y_vals[0]):
        Z = np.flipud(Z); y_vals = y_vals[::-1]
    return x_vals, y_vals, Z

def _affine_from_axes(x_vals: np.ndarray, y_vals: np.ndarray) -> Affine:
    dx = _median_step(x_vals)
    dy = _median_step(y_vals)
    x0 = float(x_vals.min()); y0 = float(y_vals.max())
    return Affine.translation(x0, y0) * Affine.scale(dx, -dy)

def _write_geotiff(out_tif: Path, Z: np.ndarray, transform: Affine, crs: CRS) -> None:
    out_tif.parent.mkdir(parents=True, exist_ok=True)
    profile = {
        "driver": "GTiff", "height": Z.shape[0], "width": Z.shape[1], "count": 1,
        "dtype": "float32", "crs": crs, "transform": transform, "nodata": NODATA,
        "tiled": False, "compress": "LZW", "predictor": 1, "interleave": "pixel",
    }
    with rasterio.open(out_tif, "w", **profile) as dst:
        dst.write(np.where(np.isfinite(Z), Z, NODATA).astype("float32"), 1)

def reproject_to_wgs84(src_tif: Path, dst_tif: Path, dst_res: Optional[float] = None) -> None:
    with rasterio.open(src_tif) as src:
        dst_crs = CRS.from_epsg(4326)
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds, resolution=dst_res
        )
        profile = src.profile.copy()
        profile.update(crs=dst_crs, transform=transform, width=width, height=height,
                       dtype="float32", nodata=profile.get("nodata", NODATA),
                       tiled=False, compress="LZW", predictor=1, interleave="pixel")
        data = src.read(1)

    dst_tif.parent.mkdir(parents=True, exist_ok=True)
    with rasterio.open(dst_tif, "w", **profile) as dst:
        out = np.empty((height, width), dtype="float32")
        reproject(
            data, out,
            src_transform=src.transform, src_crs=src.crs,
            dst_transform=transform,   dst_crs=CRS.from_epsg(4326),
            resampling=Resampling.bilinear,
            src_nodata=profile["nodata"], dst_nodata=profile["nodata"],
        )
        dst.write(out, 1)

def bbox_overlap(a: Tuple[float, float, float, float], b: Tuple[float, float, float, float]) -> float:
    # (xmin, xmax, ymin, ymax)
    ixmin, ixmax = max(a[0], b[0]), min(a[1], b[1])
    iymin, iymax = max(a[2], b[2]), min(a[3], b[3])
    iw = max(0.0, ixmax - ixmin); ih = max(0.0, iymax - iymin)
    inter = iw * ih
    area_a = (a[1] - a[0]) * (a[3] - a[2])
    area_b = (b[1] - b[0]) * (b[3] - b[2])
    union = area_a + area_b - inter if (area_a + area_b - inter) > 0 else 1.0
    return inter / union

def candidate_crs_list() -> List[CRS]:
    # 1) UTM zone 20N (WGS84)
    c1 = CRS.from_proj4("+proj=utm +zone=20 +datum=WGS84 +units=m +no_defs")
    # 2) UTM zone 20N (NAD27)
    c2 = CRS.from_proj4("+proj=utm +zone=20 +datum=NAD27 +units=m +no_defs")
    # 3) British West Indies Grid (Clarke 1880), lon0=-62, k=0.9995, FE=400000
    c3 = CRS.from_proj4(
        "+proj=tmerc +lat_0=0 +lon_0=-62 +k=0.9995 +x_0=400000 +y_0=0 "
        "+a=6378249.145 +b=6356514.86955 +units=m +no_defs"
    )
    # 4) British West Indies Grid (International 1924)
    c4 = CRS.from_proj4(
        "+proj=tmerc +lat_0=0 +lon_0=-62 +k=0.9995 +x_0=400000 +y_0=0 "
        "+a=6378388 +b=6356911.946 +units=m +no_defs"
    )
    return [c1, c2, c3, c4]

def detect_elevation_units(z: np.ndarray) -> str:
    zmax = np.nanmax(z)
    return "feet" if zmax > 2000 else "meters"

def scale_xy_about_lower_left(x: np.ndarray, y: np.ndarray, s: float) -> Tuple[np.ndarray, np.ndarray]:
    x0, y0 = float(np.min(x)), float(np.min(y))
    return x0 + s * (x - x0), y0 + s * (y - y0)

def test_combo(e: np.ndarray, n: np.ndarray, z: np.ndarray, scale: float, crs: CRS, label: str,
               out_dir: Path, default_region) -> Tuple[float, Path, Path]:
    xs, ys = scale_xy_about_lower_left(e, n, scale)
    x_vals, y_vals, Z = _grid_xyz(xs, ys, z)
    T = _affine_from_axes(x_vals, y_vals)
    native_tif = out_dir / f"native_{label}.tif"
    wgs84_tif  = out_dir / f"wgs84_{label}.tif"
    _write_geotiff(native_tif, Z, T, crs)
    reproject_to_wgs84(native_tif, wgs84_tif)
    with rasterio.open(wgs84_tif) as src:
        b = src.bounds
    dem_region = (b.left, b.right, b.bottom, b.top)
    score = bbox_overlap(dem_region, default_region)
    print(f"[{label}] WGS84 bounds {np.round(dem_region, 6).tolist()}  overlap={score:.3f}")
    return score, native_tif, wgs84_tif

def plot_overlay(wgs84_tif: Path, png_out: Path, default_region):
    if not PLOT_OVERLAY:
        print("PyGMT not available; skipping overlay plot.")
        return
    fig = pygmt.Figure()
    fig.grdimage(grid=str(wgs84_tif), region=default_region, projection="M12c",
                 cmap="geo", shading=True, frame=["af", "+tDEM (best candidate)"])
    fig.savefig(png_out)
    print(f"✅ Wrote overlay plot: {png_out}")


## 1) Fit CRS and XY Scale
Runs through the candidate CRSs and scale factors:
`[1.0, 0.4, 0.3048] × [c1, c2, c3, c4]`, scoring the WGS84 bounding box overlap.

In [4]:

# Run the search
OUT_DIR.mkdir(parents=True, exist_ok=True)

arr = np.loadtxt(ASC_XYZ, dtype=float)
assert arr.shape[1] >= 3, "Expected 3 columns: easting northing elevation"
e, n, z = arr[:, 0], arr[:, 1], arr[:, 2]

# Elevation units check (convert to meters if likely feet)
units = detect_elevation_units(z)
if units == "feet":
    print("⚠️ Elevations look like feet; converting to meters.")
    z = z * 0.3048

best = None  # (score, label, native_tif, wgs84_tif, crs, s)
for s in SCALE_FACTORS:
    for i, crs in enumerate(candidate_crs_list(), 1):
        label = f"s{s}_{i}"
        score, native_tif, wgs84_tif = test_combo(e, n, z, s, crs, label, OUT_DIR, DEFAULT_REGION)
        if (best is None) or (score > best[0]):
            best = (score, label, native_tif, wgs84_tif, crs, s)

if best is None:
    raise RuntimeError("No combination produced a valid raster.")

score, label, native_tif, wgs84_tif, best_crs, best_scale = best
print("\n🏁 Best combination:")
print(f"  scale={best_scale}, CRS={best_crs.to_string()}")
print(f"  native: {native_tif}")
print(f"  wgs84 : {wgs84_tif}")
print(f"  overlap score: {score:.3f}")

# Optional overlay
if PLOT_OVERLAY:
    plot_overlay(wgs84_tif, OUT_DIR / f"overlay_best_{label}.png", DEFAULT_REGION)


[s1.0_1] WGS84 bounds [-64.192075, -63.90824, 16.661296, 17.069551]  overlap=0.000
[s1.0_2] WGS84 bounds [-64.192064, -63.90822, 16.662317, 17.070586]  overlap=0.000
[s1.0_3] WGS84 bounds [-62.252366, -61.970322, 16.667854, 17.074793]  overlap=0.174
[s1.0_4] WGS84 bounds [-62.252359, -61.970335, 16.666183, 17.073093]  overlap=0.176
[s0.4_1] WGS84 bounds [-64.190539, -64.077029, 16.661565, 16.824832]  overlap=0.000
[s0.4_2] WGS84 bounds [-64.190529, -64.077015, 16.662581, 16.825854]  overlap=0.000
[s0.4_3] WGS84 bounds [-62.252041, -62.139244, 16.667932, 16.830769]  overlap=0.850
[s0.4_4] WGS84 bounds [-62.252034, -62.139246, 16.666267, 16.829092]  overlap=0.850
[s0.3048_1] WGS84 bounds [-64.190297, -64.103806, 16.661518, 16.785992]  overlap=0.000
[s0.3048_2] WGS84 bounds [-64.190287, -64.103793, 16.662533, 16.787012]  overlap=0.000
[s0.3048_3] WGS84 bounds [-62.25199, -62.166041, 16.66796, 16.792038]  overlap=0.494
[s0.3048_4] WGS84 bounds [-62.251983, -62.166041, 16.666295, 16.790364]

## 2) Remove Small Offshore Islands
Mask positive-elevation components (≥ sea level) smaller than a minimum area.

**Tip:** on your dataset sea level thresholds around `0.1–0.3 m` worked best.

In [5]:

from scipy import ndimage

def _pixel_meters(transform: Affine, crs: CRS, lat_hint: float | None) -> tuple[float, float]:
    """Return (dx_m, dy_m) for one pixel."""
    dx = abs(transform.a); dy = abs(transform.e)
    if crs and crs.is_geographic:
        lat = 0.0 if lat_hint is None else float(lat_hint)
        m_per_deg_lat = 111_132.0
        m_per_deg_lon = 111_320.0 * np.cos(np.deg2rad(lat))
        return dx * m_per_deg_lon, dy * m_per_deg_lat
    else:
        return dx, dy

def remove_islands(
    dem_in: Path,
    dem_out: Path,
    sea_level: float = 0.0,
    min_island_km2: float = 0.2,
    keep_largest_always: bool = True,
    set_removed_to: float = 0.0,    # set small islands to sea (or np.nan)
) -> Path:
    dem_in = Path(dem_in); dem_out = Path(dem_out)
    with rasterio.open(dem_in) as src:
        z = src.read(1, masked=False).astype("float32")
        prof = src.profile.copy()
        tr = src.transform
        crs = src.crs
        ny = src.height
        y0 = tr.f + tr.e * (0 + 0.5)
        y1 = tr.f + tr.e * (ny - 1 + 0.5)
        lat_hint = (y0 + y1) / 2.0

    finite = np.isfinite(z)
    land = finite & (z >= sea_level)
    if not np.any(land):
        raise RuntimeError("No land pixels found at/above the sea_level.")

    structure = np.ones((3,3), dtype=np.uint8)
    labels, nlab = ndimage.label(land, structure=structure)

    dx_m, dy_m = _pixel_meters(tr, crs, lat_hint)
    pix_area_m2 = max(dx_m * dy_m, 1e-6)
    min_pix = int(np.ceil((min_island_km2 * 1_000_000.0) / pix_area_m2))

    sizes = np.bincount(labels.ravel())
    largest_label = 1 + int(np.argmax(sizes[1:])) if nlab > 0 else 0

    keep = np.zeros_like(land, dtype=bool)
    if keep_largest_always and largest_label > 0:
        keep |= (labels == largest_label)
    for lbl in range(1, nlab + 1):
        if lbl == largest_label and keep_largest_always:
            continue
        if sizes[lbl] >= min_pix:
            keep |= (labels == lbl)

    drop = land & (~keep)

    cleaned = z.copy()
    cleaned[drop] = (np.nan if np.isnan(set_removed_to) else float(set_removed_to))

    dem_out.parent.mkdir(parents=True, exist_ok=True)
    with rasterio.open(dem_out, "w", **prof) as dst:
        dst.write(cleaned.astype("float32"), 1)

    km2_dropped = (sizes[1:].sum() - sizes[largest_label]) * pix_area_m2 / 1_000_000.0 if keep_largest_always and nlab>0 else 0.0
    print(f"✓ Saved cleaned DEM → {dem_out}")
    print(f"   Pixel ~ {dx_m:.1f}×{dy_m:.1f} m; min area = {min_island_km2:.3f} km² (~{min_pix} px)")
    print(f"   Components: {nlab}; dropped-small-land ≈ {km2_dropped:.3f} km²")
    return dem_out


In [6]:

# --- Example: clean the best WGS84 candidate ---
best_wgs84 = OUT_DIR / f"wgs84_{label}.tif"   # from section 1
clean_out  = OUT_DIR / f"wgs84_{label}_clean.tif"

# Sea level and min area tuned from your trials
SEA_LEVEL = 0.3    # meters
MIN_KM2   = 60.0   # km^2
KEEP_LARGEST = True

_ = remove_islands(best_wgs84, clean_out, sea_level=SEA_LEVEL,
                   min_island_km2=MIN_KM2, keep_largest_always=KEEP_LARGEST,
                   set_removed_to=0.0)
clean_out


✓ Saved cleaned DEM → /Users/glennthompson/Dropbox/PROFESSIONAL/DATA/wadgeDEMs/auto_crs_fit_v2/wgs84_s0.4_3_clean.tif
   Pixel ~ 9.8×10.2 m; min area = 60.000 km² (~605204 px)
   Components: 85; dropped-small-land ≈ 7.829 km²


PosixPath('/Users/glennthompson/Dropbox/PROFESSIONAL/DATA/wadgeDEMs/auto_crs_fit_v2/wgs84_s0.4_3_clean.tif')